# Optimización del despacho de buses en el servicio de transporte público de Monteria

Este cuaderno presenta un modelo de flujo en redes basado en programación lineal entera mixta (MILP) para despachar una flota de buses. El objetivo es satisfacer la demanda de pasajeros a lo largo de una ruta con dos cabeceras y un patio, mientras se minimiza la demanda no satisfecha.

La operación se modela como un flujo en una red espacio-temporal, donde los nodos representan ubicaciones en momentos específicos y los arcos representan posibles movimientos de los buses.

---


In [28]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

## Parámetros iniciales
Aquí se definen los parámetros iniciales del modelo, incluyendo la capacidad del bus, el intervalo de tiempo entre despachos, y las ubicaciones posibles (cabeceras y patio).
1. **Capacidad del bus (C):** Cada bus puede transportar hasta 10 pasajeros.
2. **Intervalo de tiempo (Δt):** El tiempo está dividido en intervalos de 10 minutos.
3. **Ubicaciones (L):** Los buses operan entre dos cabeceras (Cabecera 1, Cabecera 2) y un patio.
4. **Demanda:** Se genera una demanda para cada cabecera en cada intervalo de tiempo.

In [29]:
# Parámetros iniciales
C = 10  # Capacidad del bus
delta_t = 10  # Intervalo de tiempo en minutos
hora_inicio = datetime.strptime("05:00:00", "%H:%M:%S")
hora_fin = datetime.strptime("21:00:00", "%H:%M:%S")
intervalos = int((hora_fin - hora_inicio).total_seconds() // (delta_t * 60))  # Total de intervalos
T = list(range(intervalos))  # Índices de intervalos de tiempo
B = 5  # Número total de buses
L = ['Cabecera1', 'Cabecera2', 'Patio']  # Ubicaciones

## Tiempos de viaje y creación de arcos
Los tiempos de viaje entre cabeceras y entre intervalos consecutivos se definen aquí. Además, se crean los arcos que representan las posibles acciones de los buses en el tiempo-espacio (espera, viaje entre cabeceras, inicio y fin en el patio).

In [30]:
# Tiempos de viaje en intervalos
T_12 = int(round(30 / delta_t))  # Intervalos para Cabecera1 -> Cabecera2
T_21 = int(round(33 / delta_t))  # Intervalos para Cabecera2 -> Cabecera1

# Crear lista de arcos
A_arcs = []

# Arcos de espera en la misma ubicación
for l in L:
    for t in T[:-1]:  # No podemos esperar en el último intervalo
        A_arcs.append((l, t, l, t+1))

# Arcos de viaje entre cabeceras
for t in T:
    if t + T_12 < len(T):
        A_arcs.append(('Cabecera1', t, 'Cabecera2', t + T_12))
    if t + T_21 < len(T):
        A_arcs.append(('Cabecera2', t, 'Cabecera1', t + T_21))

# Arcos de inicio desde el patio (suponemos tiempo de traslado cero)
for t in T:
    A_arcs.append(('Patio', t, 'Cabecera1', t))
    A_arcs.append(('Patio', t, 'Cabecera2', t))

# Arcos de retorno al patio al finalizar operación
for t in T:
    A_arcs.append(('Cabecera1', t, 'Patio', t))
    A_arcs.append(('Cabecera2', t, 'Patio', t))


## Generación de demanda pronosticada
La demanda para cada cabecera en cada intervalo de tiempo es generada utilizando una distribución de Poisson(en este caso, mas adelante se puede incorporar el modelo de predicción de Machine Learning). Esto simula la llegada de pasajeros a las estaciones.


In [31]:
# Demanda pronosticada en cada cabecera
demanda_predicha_1 = np.random.poisson(lam=10, size=intervalos)
demanda_predicha_2 = np.random.poisson(lam=12, size=intervalos)

# Crear DataFrame de demanda
demanda_df = pd.DataFrame({
    'Tiempo': T,
    'Demanda_Cabecera1': demanda_predicha_1[:intervalos],
    'Demanda_Cabecera2': demanda_predicha_2[:intervalos]
})

## Definición del modelo y variables de decisión
Se define el modelo, donde las variables de decisión indican si un bus se mueve de una ubicación a otra en un intervalo de tiempo. También se introduce una variable continua para modelar la demanda no satisfecha.


In [32]:
# Crear el modelo
m = gp.Model("DespachoBuses")

# Conjunto de buses
K = list(range(B))

# Variables de decisión
x = m.addVars(
    K,
    A_arcs,
    vtype=GRB.BINARY,
    name="x"
)
y = m.addVars(L[:2], T, vtype=GRB.CONTINUOUS, name="y")  # Solo en cabeceras


## Función objetivo: minimizar la demanda no satisfecha
La función objetivo busca minimizar la cantidad total de demanda no satisfecha en ambas cabeceras.


In [33]:
# Función objetivo: Minimizar demanda no satisfecha
m.setObjective(gp.quicksum(y[l, t] for l in L[:2] for t in T), GRB.MINIMIZE)

# Definir nodos de inicio y fin para cada bus
nodo_inicio_bus = {}
nodo_fin_bus = {}
for k in K:
    nodo_inicio_bus[k] = ('Patio', 0)  # Inicia en el patio en el primer intervalo
    nodo_fin_bus[k] = ('Patio', intervalos - 1)  # Termina en el patio en el último intervalo

### Restricciones de satisfacción de la demanda
Cada cabecera debe cubrir la demanda de pasajeros en cada intervalo de tiempo. La capacidad total de los buses más la demanda no satisfecha debe ser mayor o igual a la demanda pronosticada.


In [34]:
for l in L[:2]:  # Solo para cabeceras
    for t in T:
        demanda = demanda_df.loc[t, 'Demanda_Cabecera1'] if l == 'Cabecera1' else demanda_df.loc[t, 'Demanda_Cabecera2']
        flujo_salida = gp.quicksum(
            C * x[k, l, t, l_to, t_to]
            for k in K
            for (l_from, t_from, l_to, t_to) in A_arcs
            if l_from == l and t_from == t
        )
        m.addConstr(
            flujo_salida + y[l, t] >= demanda,
            name=f"Demanda_{l}_{t}"
        )

### Restricciones de conservación de flujo
Los flujos entrantes y salientes en cada nodo (ubicación y tiempo) deben mantenerse balanceados. Esto asegura que los buses transiten correctamente a lo largo de la red.


In [35]:
for k in K:
    for l in L:
        for t in T:
            nodo_actual = (l, t)
            flujo_entrada = gp.quicksum(
                x[k, l_prev, t_prev, l, t]
                for (l_prev, t_prev, l_curr, t_curr) in A_arcs
                if l_curr == l and t_curr == t
            )
            flujo_salida = gp.quicksum(
                x[k, l, t, l_next, t_next]
                for (l_curr, t_curr, l_next, t_next) in A_arcs
                if l_curr == l and t_curr == t
            )
            entrada_nodo_inicio = 1 if nodo_actual == nodo_inicio_bus[k] else 0
            salida_nodo_fin = 1 if nodo_actual == nodo_fin_bus[k] else 0
            m.addConstr(
                flujo_entrada + entrada_nodo_inicio == flujo_salida + salida_nodo_fin,
                name=f"Flujo_{k}_{l}_{t}"
            )


### Restricciones de inicio y fin de los buses
Cada bus debe iniciar su operación en el patio en el primer intervalo de tiempo y regresar al patio al finalizar la operación.


In [36]:
# Restricciones de inicio de los buses
for k in K:
    nodo_inicio = nodo_inicio_bus[k]
    flujo_salida_inicio = gp.quicksum(
        x[k, nodo_inicio[0], nodo_inicio[1], l_to, t_to]
        for (l_from, t_from, l_to, t_to) in A_arcs
        if l_from == nodo_inicio[0] and t_from == nodo_inicio[1]
    )
    m.addConstr(
        flujo_salida_inicio == 1,
        name=f"Inicio_{k}"
    )

# Restricciones de fin de los buses
for k in K:
    nodo_fin = nodo_fin_bus[k]
    flujo_entrada_fin = gp.quicksum(
        x[k, l_from, t_from, nodo_fin[0], nodo_fin[1]]
        for (l_from, t_from, l_to, t_to) in A_arcs
        if l_to == nodo_fin[0] and t_to == nodo_fin[1]
    )
    m.addConstr(
        flujo_entrada_fin == 1,
        name=f"Fin_{k}"
    )


### Resolución del modelo
Finalmente, se resuelve el modelo utilizando Gurobi.


In [37]:
# Resolver el modelo
m.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (22631.2))

CPU model: Intel(R) Core(TM) i5-9300H CPU @ 2.40GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1642 rows, 4467 columns and 11612 nonzeros
Model fingerprint: 0x6fac6cf7
Variable types: 192 continuous, 4275 integer (4275 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Presolve removed 30 rows and 40 columns
Presolve time: 0.02s
Presolved: 1612 rows, 4427 columns, 11507 nonzeros
Variable types: 0 continuous, 4427 integer (4334 binary)

Root relaxation: objective 0.000000e+00, 3703 iterations, 0.24 seconds (0.23 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.00000    0 

## Resultados y visualización
Se extraen los resultados óptimos, calculando la demanda no satisfecha y generando el programa de despacho para cada bus.


In [38]:
if m.status == GRB.OPTIMAL:
    # Demanda no satisfecha total
    demanda_no_satisfecha_total = sum(y[l, t].X for l in L[:2] for t in T)
    print(f"Demanda no satisfecha total: {demanda_no_satisfecha_total}")
    
    # Programa de Despacho de Buses
    bus_schedule = []
    
    for k in K:
        for (l_from, t_from, l_to, t_to) in A_arcs:
            if x[k, l_from, t_from, l_to, t_to].X > 0.5:
                hora_inicio_arco = hora_inicio + timedelta(minutes=t_from * delta_t)
                hora_fin_arco = hora_inicio + timedelta(minutes=t_to * delta_t)
                bus_schedule.append({
                    'Bus': k,
                    'Desde': l_from,
                    'Hasta': l_to,
                    'Hora Inicio': hora_inicio_arco.strftime("%H:%M"),
                    'Hora Fin': hora_fin_arco.strftime("%H:%M")
                })
    
    bus_schedule_df = pd.DataFrame(bus_schedule)
    print("\nPrograma de Despacho de Buses:")
    print(bus_schedule_df)
    
    # Cálculo de Métricas de Desempeño
    # Demanda total y atendida
    demanda_total = demanda_df['Demanda_Cabecera1'].sum() + demanda_df['Demanda_Cabecera2'].sum()
    demanda_atendida = demanda_total - demanda_no_satisfecha_total
    print(f"\nDemanda total: {demanda_total}")
    print(f"Demanda atendida: {demanda_atendida}")
    
    # Kilómetros recorridos e IPK
    distancia_cabeceras = 20  # km (ajusta según corresponda)
    kilometros_recorridos = 0
    for index, row in bus_schedule_df.iterrows():
        if (row['Desde'] == 'Cabecera1' and row['Hasta'] == 'Cabecera2') or (row['Desde'] == 'Cabecera2' and row['Hasta'] == 'Cabecera1'):
            kilometros_recorridos += distancia_cabeceras
    
    ipk = demanda_atendida / kilometros_recorridos if kilometros_recorridos > 0 else 0
    print(f"Kilómetros recorridos: {kilometros_recorridos}")
    print(f"IPK: {ipk}")
else:
    print("No se encontró una solución óptima.")


Demanda no satisfecha total: 0.0

Programa de Despacho de Buses:
      Bus      Desde      Hasta Hora Inicio Hora Fin
0       0  Cabecera1  Cabecera1       05:20    05:30
1       0  Cabecera1  Cabecera1       05:30    05:40
2       0  Cabecera1  Cabecera1       07:00    07:10
3       0  Cabecera1  Cabecera1       08:30    08:40
4       0  Cabecera1  Cabecera1       09:00    09:10
...   ...        ...        ...         ...      ...
1106    4  Cabecera2      Patio       20:00    20:00
1107    4  Cabecera1      Patio       20:10    20:10
1108    4  Cabecera2      Patio       20:20    20:20
1109    4  Cabecera1      Patio       20:40    20:40
1110    4  Cabecera2      Patio       20:40    20:40

[1111 rows x 5 columns]

Demanda total: 2040
Demanda atendida: 2040.0
Kilómetros recorridos: 720
IPK: 2.8333333333333335
